<a href="https://colab.research.google.com/github/kazimianec/TARIC/blob/master/Training_TRCtransformersSBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch


In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
print(torch.cuda.device_count())

1


In [4]:
url_TARIC_C2_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C2_descriptions.csv"
url_TARIC_C4_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C4_descriptions.csv"
url_TARIC_C6_descrptions ="https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C6_descriptions.csv"
url_TARIC_C246_descrptions = "https://raw.githubusercontent.com/kazimianec/TARIC/master/DATA/TARIC_C246_descriptions_CONCAT.csv"


In [5]:
import pandas as pd 
import io 
  
df_C2_descriptions = pd.read_csv(url_TARIC_C2_descrptions) 
df_C4_descriptions = pd.read_csv(url_TARIC_C4_descrptions)
df_C6_descriptions = pd.read_csv(url_TARIC_C6_descrptions)
df_C246_descriptions = pd.read_csv(url_TARIC_C246_descrptions)

In [6]:
for col in df_C246_descriptions.columns: 
    print(col)

CODE_2
CODE_4
CODE_6
goods_CODE_6
Description_46
Description_246
Description_46_concat
Description_246_concat


In [7]:
#sentences = df['CLASS_Description'].tolist()

sentences_C2 = df_C2_descriptions['Description_2'].tolist()
sentences_C4 = df_C4_descriptions['Description_4'].tolist()
sentences_C6 = df_C6_descriptions['Description_6'].tolist()

sentences_C246 = df_C246_descriptions['Description_246'].tolist()
sentences_C46 = df_C246_descriptions['Description_46'].tolist()


In [8]:
!pip install sentence_transformers

In [9]:
from sentence_transformers import SentenceTransformer, util


**SBERT**
Models can be loaded from SBERT , full list:
https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/edit#gid=0


In [32]:
#model = SentenceTransformer('average_word_embeddings_glove.840B.300d')
#model = SentenceTransformer('paraphrase-distilroberta-base-v1') --no
#model= SentenceTransformer('msmarco-distilroberta-base-v2') --no
#model = SentenceTransformer('stsb-bert-large')  ##maybe
##model = SentenceTransformer('stsb-roberta-large') --no
#model= SentenceTransformer('paraphrase-xlm-r-multilingual-v1')
model= SentenceTransformer('LaBSE')  # interesting
#model = SentenceTransformer('quora-distilbert-multilingual') ## not ok

#model = SentenceTransformer('average_word_embeddings_glove.840B.300d')

In [33]:
sentences = sentences_C246
sentences = [element.lower() for element in sentences]


In [34]:

embeddings = model.encode(sentences, convert_to_tensor=True)
#embeddings_246_concat = model.encode(sentences_246_concat, convert_to_tensor=True)
#embeddings_46_concat = model.encode(sentences_46_concat, convert_to_tensor=True)
#embeddings_246 = model.encode(sentences_246, convert_to_tensor=True)
#embeddings_46 = model.encode(sentences_46, convert_to_tensor=True)

In [35]:
TAROC_cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
print(TAROC_cosine_scores.shape)

top_k=5
top_results = torch.topk(TAROC_cosine_scores, k=top_k)
print(TAROC_cosine_scores.shape)
print(TAROC_cosine_scores)
print(top_results.values)
print(top_results.indices)
#print(sentences[0])
#print(sentences[40])
#y = torch.ones(TAROC_cosine_scores.shape)*0.8
#torch.where(TAROC_cosine_scores > y, TAROC_cosine_scores, 0)

torch.Size([5209, 5209])
torch.Size([5209, 5209])
tensor([[1.0000, 0.9176, 0.9509,  ..., 0.3550, 0.3584, 0.4209],
        [0.9176, 1.0000, 0.9551,  ..., 0.3792, 0.3939, 0.4784],
        [0.9509, 0.9551, 1.0000,  ..., 0.3252, 0.3377, 0.3968],
        ...,
        [0.3550, 0.3792, 0.3252,  ..., 1.0000, 0.9283, 0.8807],
        [0.3584, 0.3939, 0.3377,  ..., 0.9283, 1.0000, 0.9158],
        [0.4209, 0.4784, 0.3968,  ..., 0.8807, 0.9158, 1.0000]])
tensor([[1.0000, 0.9509, 0.9176, 0.9104, 0.8976],
        [1.0000, 0.9875, 0.9551, 0.9176, 0.8996],
        [1.0000, 0.9551, 0.9509, 0.9466, 0.9013],
        ...,
        [1.0000, 0.9283, 0.8807, 0.8359, 0.7744],
        [1.0000, 0.9283, 0.9158, 0.8264, 0.7548],
        [1.0000, 0.9158, 0.8807, 0.8261, 0.7131]])
tensor([[   0,    2,    1,    4,    6],
        [   1,    3,    2,    0,   24],
        [   2,    1,    0,    3,   13],
        ...,
        [5206, 5207, 5208, 5203, 5204],
        [5207, 5206, 5208, 5204, 5205],
        [5208, 5207, 5206

In [43]:
description = ["Vyriškos kelnės"]

description = ["Мужские джинсы"]
description = ["Beef fillet"]

description = ["Jautienos filė"]

description = ["Лак для маникюра"]

description = ["Nail polish"]

description = ["Кресла офисные"]

description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]

description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["I Love Makeup - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]



description = ["папиросы"]
description = ["I Love Makeup  - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Beef fillet"]
description = ["Гoвядина филе морож"]
description = ["Мужские джинсы"]
description = ["Фанера"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Vineer on kihiline materjal, mis valmistatakse õhukeste puitlehtede – spoonide – kokkuliimimise teel."] 
description = ["офисные стулья деревянные дубовые"] 
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Fosmon Apple iPhone 4 / iPhone 4G USB Sync Charge Data Cable with USB Auto Car Charger and USB Home Travel Charger"] #SUPER matches S3
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"] # not
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"] ## bullshit is both cases - bert and S3
description = ["Jolly Jumper Pashmama Nursing Cover - Tan"]
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["I Love Makeup  - Nail Polish"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["HTC HD7 Crowned Heart Phone Protector Cover Case"]


description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Beef fillet"]
description = ["Astro Pneumatic 1716 Deluxe Air Blow Gun with 20-Inch Long Angled Nozzle and 1/2-Inch Removable Rubber Tip"]
description = ["Phone Protector Cover Case"]
description = ["I Love Makeup  - Nail Polish"]

embeddings_description = model.encode(description, convert_to_tensor=True )

In [44]:
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings_description)
cosine_scores.shape
values,indices = torch.transpose(cosine_scores,0,1).topk(10)
print(values, indices)
indices.numpy()[0][0]

print(values.numpy()[0][0], sentences[indices.numpy()[0][0]])
print(values.numpy()[0][1], sentences[indices.numpy()[0][1]])
print(values.numpy()[0][2], sentences[indices.numpy()[0][2]])
print(values.numpy()[0][3], sentences[indices.numpy()[0][3]])
print(values.numpy()[0][4], sentences[indices.numpy()[0][4]])




tensor([[0.2851, 0.2823, 0.2696, 0.2674, 0.2573, 0.2453, 0.2394, 0.2289, 0.2262,
         0.2229]]) tensor([[1680, 1681, 1682, 1683, 1684, 5071, 5069, 5070, 3830, 1692]])
0.28513616 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; lip make-up preparations
0.28226382 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; eye make-up preparations
0.26960486 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure prep

In [45]:
df_C246_descriptions.iloc[indices.numpy()[0],:]

,CODE_2,CODE_4,CODE_6,goods_CODE_6,Description_46,Description_246,Description_46_concat,Description_246_concat
1680,33,4,10,3304100000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
1681,33,4,20,3304200000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
1682,33,4,30,3304300000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
1683,33,4,91,3304910000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
1684,33,4,91,3304990000 80,Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...",Beauty or make-up preparations and preparation...,"ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."
5071,96,3,30,9603300000 80,"Brooms, brushes (including brushes constitutin...","MISCELLANEOUS MANUFACTURED ARTICLES; Brooms, b...","Brooms, brushes (including brushes constitutin...","MISCELLANEOUS MANUFACTURED ARTICLES; Brooms, b..."
5069,96,3,21,9603210000 80,"Brooms, brushes (including brushes constitutin...","MISCELLANEOUS MANUFACTURED ARTICLES; Brooms, b...","Brooms, brushes (including brushes constitutin...","MISCELLANEOUS MANUFACTURED ARTICLES; Brooms, b..."
5070,96,3,21,9603290000 80,"Brooms, brushes (including brushes constitutin...","MISCELLANEOUS MANUFACTURED ARTICLES; Brooms, b...","Brooms, brushes (including brushes constitutin...","MISCELLANEOUS MANUFACTURED ARTICLES; Brooms, b..."
3830,82,14,20,8214200000 80,"Other articles of cutlery (for example, hair c...","TOOLS, IMPLEMENTS, CUTLERY, SPOONS AND FORKS, ...","Other articles of cutlery (for example, hair c...","TOOLS, IMPLEMENTS, CUTLERY, SPOONS AND FORKS, ..."
1692,33,7,10,3307100000 80,"Pre-shave, shaving or aftershave preparations,...","ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME...","Pre-shave, shaving or aftershave preparations,...","ESSENTIAL OILS AND RESINOIDS; PERFUMERY, COSME..."


**TRAINING**

In [47]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
t_model = model

#Define your train examples. You need more than just two examples...
#train_examples = [
#                  InputExample(texts=['Nail polish makeup', 'manicure or pedicure preparations'], label=1.),
#                  InputExample(texts=['Nail polish makeup', 'essential oils and resinoids; perfumery, cosmetic or toilet preparations'], label=1.),
#                  InputExample(texts=['Nail polish makeup', 'soap, organic surface-active agents, washing preparations, lubricating preparations, artificial waxes, prepared waxes, polishing or scouring preparations, candles and similar articles, modelling pastes'], label=0.)]

train_examples = [
                  InputExample(texts=['Nail polish makeup', 'manicure or pedicure preparations'], label=1.),
                  InputExample(texts=['Nail polish makeup', 'essential oils and resinoids; perfumery, cosmetic or toilet preparations'], label=1.),
                  InputExample(texts=['Nail polish makeup', 'lips hair eye'], label=0.)]



#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=1)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
t_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

RuntimeError: ignored

In [52]:
t_embeddings = t_model.encode(sentences, convert_to_tensor=True)

In [53]:
t_embeddings_description = t_model.encode(description, convert_to_tensor=True )

cosine_scores = util.pytorch_cos_sim(t_embeddings, t_embeddings_description)
cosine_scores.shape
values,indices = torch.transpose(cosine_scores,0,1).topk(10)
print(values, indices)
indices.numpy()[0][0]

print(values.numpy()[0][0], sentences[indices.numpy()[0][0]])
print(values.numpy()[0][1], sentences[indices.numpy()[0][1]])
print(values.numpy()[0][2], sentences[indices.numpy()[0][2]])
print(values.numpy()[0][3], sentences[indices.numpy()[0][3]])
print(values.numpy()[0][4], sentences[indices.numpy()[0][4]])



tensor([[0.5125, 0.4938, 0.4717, 0.4644, 0.4457, 0.4372, 0.4114, 0.3933, 0.3931,
         0.3917]]) tensor([[1680, 1713, 1681, 1712, 1682, 1684, 1683, 1649, 1689, 1698]])
0.51250577 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan preparations; manicure or pedicure preparations; lip make-up preparations
0.49377793 soap, organic surface-active agents, washing preparations, lubricating preparations, artificial waxes, prepared waxes, polishing or scouring preparations, candles and similar articles, modelling pastes, 'dental waxes' and dental preparations with a basis of plaster; artificial waxes and prepared waxes; other
0.47173536 essential oils and resinoids; perfumery, cosmetic or toilet preparations; beauty or make-up preparations and preparations for the care of the skin (other than medicaments), including sunscreen or suntan prepar